In [1]:
from langdetect import detect, detect_langs
import csv
import sys

csv.field_size_limit(sys.maxsize)

131072

In [2]:
def readCsvFile(fileName: str) -> dict:
    data = []
    with open(fileName, newline='', encoding="utf-8-sig") as file:
        spamreader = csv.DictReader(file, fieldnames=['id', 'native', 'language', 'text'])
        for row in spamreader:
            row['text'] = ' '.join(row['text'].splitlines())
            row['text'].strip()
            data.append(row)
    return data

In [3]:
data = readCsvFile('./data/dataset_cleand-1.csv')

In [4]:
# takes about 3 mins to run
new_data = []
counter = 1000
for entry in data:
    try:
        detected_langs = detect_langs(entry['text'])
        for lang in detected_langs:
            if lang.lang == 'en' and lang.prob > 0.5:
                new_data.append(entry)
                '''
                print(entry['text'])
                print(detected_langs)
                '''
                new_data[len(new_data)- 1]["probabilities"] = detected_langs 
                if lang.prob >= 0.99:
                    new_data[len(new_data)- 1]['english_only'] = True
                else:
                    new_data[len(new_data) - 1]['english_only'] = False
    except:
        print("weird entry")
        print(entry)

weird entry
{'id': '7324', 'native': 'Arabic', 'language': "[{'language': 'English', 'level': 45, 'proficiency': 'Intermediate'}, {'language': 'Japanese', 'level': 19.285714285714285, 'proficiency': 'Elementary'}]", 'text': 'I love Japan♡♡♡♡♡♡♡ ♡♡♡♡♡♡♡♡♡♡♡♡♡♡♡♡♡♡♡♡♡♡♡♡♡♡♡♡♡'}
weird entry
{'id': '58860', 'native': 'Persian', 'language': "[{'language': 'English', 'level': 45, 'proficiency': 'Intermediate'}]", 'text': 'ᴄᴏғғᴇᴇ ᴀᴅᴅɪᴄᴛ ◑ᴸᴼᴼᴷᴵᴺᴳ ᶠᴼᴿ ᴬ ᴳᴼᴼᴰ ᴮᴼᴼᴷ, ᵐᵒᵛⁱᵉ&amp; ˢᵉʳⁱᵉ ✯ᴘᴀʀᴛ-ᴛɪᴍᴇ ɢᴀᴍᴇʀ ✤ᴏᴠᴏ-ʟᴀᴄᴛᴏ ᴠᴇɢᴇᴛᴀʀɪᴀɴɪsᴍ'}


In [ ]:
from copy import deepcopy

new_data_copy = deepcopy(new_data)

In [ ]:
def writeToCsv(filename: str) -> dict:
    with open(filename, 'w') as f:
        writer = csv.DictWriter(f, fieldnames=['id', 'native', 'language', 'probabilities', 'english_only', 'text'])
        writer.writeheader()
        writer.writerows(new_data_copy)

writeToCsv("./data/new_data_utf.csv")

In [ ]:
from spacy.language import Language
from spacy_langdetect import LanguageDetector
import spacy

def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

In [8]:
from langdetect import detect, DetectorFactory, detect_langs
my_string = "Hi, my friend lives next to me. Can you call her? Thibault François. Envoyé depuis mon mobile"

DetectorFactory.seed = 42

# for i in range(5):
#     print(detect_langs(text_content), detect(text_content))

In [9]:
import pandas as pd
import copy

csv_file = pd.read_csv("only_english.csv", encoding='utf-8')
dataset = csv_file[['id','native','language','text']]
original_datset = copy.deepcopy(dataset)

c = 0
filtered_dataset = pd.DataFrame()

from nltk import tokenize
nltk.download('punkt')

for i in range(len(dataset['text'])):
    splitted_text = tokenize.sent_tokenize(dataset['text'][i])
    english_sent = []
    for sent in splitted_text:
        try:
            langs = detect_langs(sent)
            has_key = False
            lang_index = 0
            for l in range(len(langs)):
                if langs[l].lang == "en":
                    has_key = True
                    lang_index = l
                    break
            if has_key and langs[lang_index].prob > 0.9:
                english_sent.append(sent)
        except:
            pass
    if len(english_sent) != 0:
        dataset['text'][i] = " ".join(english_sent)
        # print(dataset['text'][i] ==  " ".join(english_sent))
        filtered_dataset = pd.concat([filtered_dataset, pd.DataFrame(dataset.iloc[[i]])])
        c += 1  

print(c)

FileNotFoundError: [Errno 2] No such file or directory: 'only_english.csv'

In [25]:
filtered_dataset.to_csv('./data/only_english2.csv')

NameError: name 'filtered_dataset' is not defined

In [26]:
text_content = "Hello. I'm looking for language partners for practice my English. I can help you with Italian of course. I wish to know new cultures. Ciao. Sto cercando dei partners linguistici per parlare inglese. Io posso aiutare a migliorare l'italiano. Desidero conoscere nuove culture."
print(detect_langs(text_content))
# print(Detector(text_content))

[en:0.8571395364815149, it:0.1428600241208517]


In [42]:
from polyglot.detect import Detector

filtered_dataset_polyg = pd.DataFrame()
c = 0
for i in range(len(dataset['text'])):
    try: 
        langss = Detector(dataset['text'][i])
        langs = langss.languages
        if dataset['text'][i] == text_content:
            print(langss)
        has_key = False
        lang_index = 0
        for l in range(len(langs)):
            if langs[l].code == "en":
                has_key = True
                lang_index = l
                break
        if has_key and langs[lang_index].confidence > 0.99:
            filtered_dataset_polyg = pd.concat([filtered_dataset_polyg, pd.DataFrame(dataset.iloc[[i]])])
            c += 1
    except:
        pass
print(c)

Detector is not able to detect the language reliably.


1890


In [18]:
print(filtered_dataset_polyg)

         id                native  \
1         2   Spanish<br/>Catalan   
4         6               Italian   
9        23   Spanish<br/>Catalan   
11       27               Spanish   
13       34               Spanish   
...     ...                   ...   
3225  32988  Spanish<br/>Galician   
3227  32990  Spanish<br/>Galician   
3228  32991  Spanish<br/>Galician   
3231  32995  Spanish<br/>Galician   
3233  32997  Spanish<br/>Galician   

                                               language  \
1     [{'language': 'English', 'level': 45, 'profici...   
4     [{'language': 'English', 'level': 45, 'profici...   
9     [{'language': 'Danish', 'level': 6.42857142857...   
11    [{'language': 'English', 'level': 45, 'profici...   
13    [{'language': 'English', 'level': 45, 'profici...   
...                                                 ...   
3225  [{'language': 'English', 'level': 45, 'profici...   
3227  [{'language': 'English', 'level': 45, 'profici...   
3228  [{'language': 'Eng

In [30]:
filtered_dataset_polyg.to_csv('./data/only_english_polyg.csv')